In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests
import time

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 2,
    "ProductOffering": 3,
    "Facility": 2,
    "Parts": 4,
    "Warehouse": 5,
    "Supplier": 5,
}

In [3]:
# API_URL = "https://bug-hardy-obviously.ngrok-free.app"
API_URL = "http://localhost:8000"

version = "dynamic_small_10_bulk_test_v6"

In [4]:
create_ops = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

print(create_ops)



Generated 287 operations
[{'action': 'create', 'type': 'schema', 'payload': {'node_id': '1', 'node_type': 'Parts', 'properties': {'id': '1', 'name': 'Name_3674', 'description': 'Description_3199', 'type': 'Type B', 'cost': 8661.67, 'importance': 1, 'expected_life': 2839, 'units_in_chain': 59, 'expiry': 59}}, 'timestamp': 0}, {'action': 'create', 'type': 'schema', 'payload': {'node_id': '1-1', 'node_type': 'Facility', 'properties': {'id': '1-1', 'name': 'Name_6273', 'type': 'Type A', 'location': 'Los Angeles', 'max_capacity': 4052, 'operating_cost': 1753.43}}, 'timestamp': 0}, {'action': 'create', 'type': 'schema', 'payload': {'source_id': '1', 'target_id': '1-1', 'edge_type': 'PartsToFacility', 'properties': {'quantity': 169, 'distance_from_warehouse': 960.79, 'transport_cost': 223.31, 'lead_time': 27}}, 'timestamp': 0}, {'action': 'create', 'type': 'schema', 'payload': {'node_id': '1-1-1', 'node_type': 'ProductOffering', 'properties': {'id': '1-1-1', 'name': 'Name_9372', 'cost': 2131.

In [5]:
bulk_create_payload = {
    "version": version,
    "action": "bulk_create",
    "type": "schema",
    "timestamp": create_ops[0]["timestamp"],
    "payload": []
}

for op in create_ops:
    bulk_create_payload["payload"].append(op["payload"])

print(bulk_create_payload)

requests.post(f"{API_URL}/schema/live/update", json=bulk_create_payload)

{'version': 'dynamic_small_10_bulk_test_v6', 'action': 'bulk_create', 'type': 'schema', 'timestamp': 0, 'payload': [{'node_id': '1', 'node_type': 'Parts', 'properties': {'id': '1', 'name': 'Name_3674', 'description': 'Description_3199', 'type': 'Type B', 'cost': 8661.67, 'importance': 1, 'expected_life': 2839, 'units_in_chain': 59, 'expiry': 59}}, {'node_id': '1-1', 'node_type': 'Facility', 'properties': {'id': '1-1', 'name': 'Name_6273', 'type': 'Type A', 'location': 'Los Angeles', 'max_capacity': 4052, 'operating_cost': 1753.43}}, {'source_id': '1', 'target_id': '1-1', 'edge_type': 'PartsToFacility', 'properties': {'quantity': 169, 'distance_from_warehouse': 960.79, 'transport_cost': 223.31, 'lead_time': 27}}, {'node_id': '1-1-1', 'node_type': 'ProductOffering', 'properties': {'id': '1-1-1', 'name': 'Name_9372', 'cost': 2131.36, 'demand': 531}}, {'source_id': '1-1', 'target_id': '1-1-1', 'edge_type': 'FacilityToProductOfferings', 'properties': {'product_cost': 446.98, 'lead_time': 24

<Response [200]>

In [6]:
update_cycles = 2
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    bulk_update_payload = {
        "version": version,
        "action": "bulk_update",
        "type": "schema",
        "timestamp": update_ops[0]["timestamp"],
        "payload": []
    }

    for op in update_ops:
        bulk_update_payload["payload"].append(op["payload"])

    print(bulk_update_payload)

    requests.post(f"{API_URL}/schema/live/update", json=bulk_update_payload)
    time.sleep(1)

Generated 19 update operations
{'version': 'dynamic_small_10_bulk_test_v6', 'action': 'bulk_update', 'type': 'schema', 'timestamp': 0, 'payload': [{'node_id': '2-2-2-2', 'node_type': 'ProductFamily', 'payload': {'id': '2-2-2-2', 'name': 'Name_8110', 'revenue': 8723.39}}, {'node_id': '1-2-1-1', 'node_type': 'ProductFamily', 'payload': {'id': '1-2-1-1', 'name': 'Name_3372', 'revenue': 5782.0}}, {'node_id': '1-2-3', 'node_type': 'ProductOffering', 'payload': {'id': '1-2-3', 'name': 'Name_2584', 'cost': 1350.09, 'demand': 964}}, {'node_id': 'Supplier_2', 'node_type': 'Supplier', 'payload': {'id': 'Supplier_2', 'name': 'Name_7822', 'location': 'Houston', 'quality': 359.94, 'reliability': 0.64, 'size': 'Medium'}}, {'node_id': 'Supplier_2', 'node_type': 'Supplier', 'payload': {'id': 'Supplier_2', 'name': 'Name_6139', 'location': 'Houston', 'quality': 359.94, 'reliability': 0.64, 'size': 'Medium'}}, {'node_id': 'Supplier_2', 'node_type': 'Supplier', 'payload': {'id': 'Supplier_2', 'name': 'Nam

In [7]:
delete_cycles = 2
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)

Generated 10 delete operations
Generated 10 delete operations
